### Challenge AbinBev
**Autor**: Mario Guzmán Briones
### Caso de Negocio
Cervecería y Maltería Quilmes cuenta con una aplicación B2B llamada BEES, donde comercializa sus productos a distintas empresas. Se realizó un estudio de satisfacción del cliente mediante encuestas con la aplicación y determinaron que uno de los puntos clave de mejora era desarrollar un nuevo módulo dentro de la aplicación que le permita a los clientes encontrar los productos más propensos a adquirir en su próximo pedido mejorando su experiencia y reduciendo el tiempo destinado a efectuar cada compra.

### Solución
El equipo de ciencia de datos de la compañía le encomiendo a usted desarrollar un modelo de Machine Learning que permita discernir que productos comprará cada uno de sus clientes en su próximo pedido para ser incluidos en el nuevo módulo de la aplicación.

### A considerar
Existe en la compañía un equipo de “Producto” que va a tomar como input el output de su modelo. El output de su modelo debe ser especificado para cada cliente y va a tener una frecuencia diaria. Este output se va a integrar en la arquitectura de datos de la app. El equipo de ingeniería se encarga de esto mediante pipelines. De esta manera el equipo de producto, va a tomar este input, le va a asignar un diseño y lo va a visualizar dentro de la aplicación como un nuevo módulo. 

### Datasets
Para el caso de estudio, se proveen los siguientes conjuntos de datos:
- **trasacciones.csv**: Este dataset contiene información transaccional de las compras realizadas por los clientes en la aplicación BEES entre finales de mayo y agosto de 2022. A continuación, se detalla el diccionario de variables de esta tabla:
    - **ACCOUNT_ID**: código de identificación del cliente.
    - **SKU_ID**: código de identificación de un producto.
    - **INVOICE_DATE**: fecha en que se realizó la transacción en formato 'yyyyMMdd'
    - **ORDER_ID**: código identificador de un pedido.
    - **ITEMS_PHYS_CASES**: cantidad de bultos que compra el cliente de un determinado SKU_ID
- **atributos.csv**: contiene atributos asociados a clientes. Las variables incluidas son:
    - **POC**: código de identificación del cliente, semánticamente, significa lo mismo que el campo "ACCOUNT_ID" en el dataset expuesto anteriormente. 
    - **BussinessSegment**: perfil digital del cliente, contiene los siguientes atributos    
        - **Variable**: referencia
        - **PowerUsage**: Indica que el cliente tiene un nivel digitalización alto, usa todo el ecosistema de la aplicación BEES
        - **HighUsage**: Indica que el cliente tiene un nivel digitalización alto, pero no usa todo el ecosistema de la aplicación BEES.
        - **MediumUsage**: Indica que tiene un nivel de uso intermedio de la aplicación en comparación con otros clientes.
        - **MinimalUsage**: Nivel de uso de la aplicación más bajo o nulo.
    - **totalVolumen**: volumen en hectolitros adquirido por el cliente en los últimos tres meses cerrados del periodo expuesto, es decir, de junio a agosto.
    - **SkuDistintosPromediosXOrden**: cantidad de productos distintos promedio adquiridos en ordenes en el periodo expuesto, es decir, de junio a agosto.
    - **SkuDistintosToTales**: cantidad de productos distintos adquiridos en el periodo expuesto, es decir, de junio a agosto.
    - **Concentración**: estimado de concentración de otros negocios de diferente índole a 150 metros cuadrados del punto de venta en cuestión, por ejemplo, un kiosco que venden Quilmes, si tiene carnicerías, tienda de ropa, panaderías ect, tiene una concentración “Alta” a su alrededor. Pueden ser Alto, Medio, Bajo o S/D que significa sin datos.
    - **nse**: nivel socioeconómico de habitantes alrededor del punto de venta. Puede ser Alto, Medio, Bajo o S/D que significa sin datos.
    - **segmentoUnico**: hace referencia a la capacidad de compra de cliente en cuestión.
        - **Inactivos**: Cliente con baja capacidad de compra
        - **Masivos**: Cliente con media capacidad de compra
        - **Potenciales**: Cliente con alta capacidad de compra 
        - **Activos**: Clientes con capacidad de compra intermedia y que, además, adquiere productos comercializados por CMQ que pertenecen a terceros.
    - **Canal**: canal de marketing al que pertenece el cliente, los valores que puede tener son: COMIDA, Tradicional, BEBIDA, Mayorista, Kioscos/Maxikioscos, OTROS REF; Instituciones, ENTRETENIMIENTO, Autoservicio, KA Minoristas,
Bar/Restaurant.


### Glosario
- CMQ -> Cervecería y Maltería Quilmes.

### Evaluación
- "EDA": La idea de este punto es que integre los dataset compartidos, se plantee una hipótesis como por ejemplo “que variables me ayudarían a realizar predicciones” y como el Feature engineering podría mejorar la performance del modelo. Desarrolle algunas visualizaciones que fundamenten dichas hipótesis.
- "Data wrangling y modelado": desarrolle el preprocesing y modelado, justifique asunciones que tome durante el proceso, y el o los modelos elegidos.
- Evalué el o los modelos desarrollados en distintos periodos, además, justifique las métricas que selecciono y el periodo de tiempo definido para la misma.
- "Output": Defina cual es el output de su modelo. Que columnas va a tener. Para esta parte, va a tener que tomar 2 definiciones: a) ¿Cuántas recomendaciones de productos va a hacer para cada cliente? ¿Cuál va a ser el criterio para definir esta cantidad?

### Librerías

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
def date_normalize(dt,inputFormat,outputFormat):
    return datetime.strptime(str(dt),inputFormat).strftime(outputFormat)

### Lectura de datos

In [10]:
path_atributos = "../data/atributos.csv"
path_transacciones = "../data/transacciones.csv"
atributos = pd.read_csv(path_atributos)
transacciones = pd.read_csv(path_transacciones)

In [11]:
atributos.drop("Unnamed: 0",axis=1,inplace=True)
transacciones.drop("Unnamed: 0",axis=1,inplace=True)

In [12]:
atributos.head()

,POC,BussinessSegment,totalVolumen,SkuDistintosPromediosXOrden,SkuDistintosToTales,concentracion,nse,segmentoUnico,canal
0,175519,HighUsage,5.18752,4.1000,16,Medio,Bajo,4.Activos,Kioscos/Maxikioscos
1,28533,HighUsage,4.76866,3.9211,34,Alto,Medio,4.Activos,Tradicional
2,32182,PowerUsage,5.97930,6.7500,34,Alto,Medio,4.Activos,Tradicional
3,327976,MinimalUsage,6.02852,3.5833,14,Alto,Medio,4.Activos,COMIDA
4,354640,PowerUsage,7.52500,3.2000,18,Bajo,S/D,4.Activos,Tradicional


In [13]:
transacciones.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ORDER_ID,ITEMS_PHYS_CASES
0,430606,7038,20220729,512-3880249-0,100.0
1,323267,14933,20220729,512-3882307-0,1.0
2,357825,21971,20220723,512-3852880-0,8.0
3,444926,7038,20220805,512-3913163-0,20.0
4,450771,7030,20220816,512-3957000-0,5.0


In [18]:
transacciones.shape

(280828, 5)

In [20]:
# Cantidad de sku únicos
transacciones["SKU_ID"].nunique()

530

In [38]:
trx_daily = transacciones\
    .groupby(["INVOICE_DATE","SKU_ID"])\
    .agg(ITEMS = ("ITEMS_PHYS_CASES","sum"))\
    .reset_index()\
    .sort_values(["INVOICE_DATE","ITEMS"],ascending=[True,False])

trx_daily["INVOICE_PERIOD"] = trx_daily["INVOICE_DATE"].apply(lambda x: date_normalize(x,"%Y%m%d","%Y%m"))

In [39]:
trx_daily\
    .groupby(["INVOICE_PERIOD","SKU_ID"])\
    .agg(ITEMS_MONTHLY = ("ITEMS","sum"))\
    .reset_index()\
    .sort_values(["INVOICE_PERIOD","SKU_ID"],ascending=[True,False])

,INVOICE_PERIOD,SKU_ID,ITEMS_MONTHLY
124,202205,77855,1.0
123,202205,77838,1.0
122,202205,77832,2.0
121,202205,77812,1.0
120,202205,77740,1.0
...,...,...,...
1001,202208,691,5.0
1000,202208,659,34.0
999,202208,641,932.0
998,202208,627,475.0
